In [1]:
from pgmpy.readwrite import BIFReader
from pathlib import Path
from src.utils import adj_df_from_BIF, get_terminal_connection_nodes, encode_data
from src.constants import alarm_target
from src.data import AlarmDataset

import pandas as pd
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split

import torch.nn as nn
import torch

In [2]:
fpath_bif = Path("/home/gaurang/bayesian_network/data/alarm/alarm.bif")
fpath_data = Path("/home/gaurang/bayesian_network/data/alarm/ALARM10k.csv")

In [3]:
df_data = pd.read_csv(fpath_data, dtype=str)
reader = BIFReader(fpath_bif)

In [4]:
data, oe = encode_data(df_data, reader)

In [5]:
dataset = AlarmDataset(data, reader)

In [7]:
dataset.adj_mat

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.]])